### Данные

In [2]:
import json
import random

In [3]:
# Открываем файл и загружаем в переменную

with open('npo_registry.json') as json_file:
    nko = json.load(json_file)

In [4]:
# Количество НКО со статусом "Зарегистрирована"

org_active = []

for x in nko[u'data']:
    if x['status'] == u'Зарегистрирована':
        org_active.append(x)

print len(org_active)

224181


In [5]:
# Количество разных форм организаций (из зарегистрированных)

forms = []
field = 'form'
for x in org_active:
    if x[field] not in forms and x[field] != '':
        forms.append(x[field])

print len(forms)

41


In [6]:
# Количество организаций без указания формы (из зарегистрированных)

form_na = []
field = 'form'
for x in org_active:
    if x[field] == '':
        form_na.append(x)

print len(form_na)

6


In [7]:
# Функция печати всех полей организации по ее индексу (в списке зарегистрированных)

def print_nko(n):
    print json.dumps(org_active[n], ensure_ascii=False, sort_keys=True, indent=4, separators=(',', ': '))

In [8]:
# Случайная НКО из зарегистрированных

n = random.randrange(len(org_active))
print n
print_nko(n)

62090
{
    "OGRN": "",
    "address": "129224 г.Москва, Студеный пр-д, д.11",
    "dateOGRN": "",
    "form": "Общественная организация",
    "org_name": "Местная общественная организация в муниципальном районе \"Северное Медведково\" Северо-Восточной окружной организации г.Москвы ЛДПР",
    "org_num": "7712014729",
    "region": "Москва",
    "registry": "Общественные объединения",
    "status": "Зарегистрирована"
}


In [9]:
# Функция поиска индекса организации в списке по ее ОГРН

def find_by_ogrn(orgs_list, ogrn):
    for i, j in enumerate(orgs_list):
        if j['OGRN'] == str(ogrn):
            return i
    return -1

### Поиск сайта организации

Предположим, что получен некоторый список сайтов, для которых нужно определить, есть ли среди них сайт данной организации.

Попробуем выделить на веб-странице то место, где обычно ставят копирайт и пишут, кому принадлежит сайт. Вероятно, что, если это сайт данной НКО, то там будет упомянуто ее название.

Для примера выберем одну организацию и попробуем найти ее сайт.

In [10]:
ogrn = 1063905027526  # Психолого-педагогическое Объединение "Азбука развития"

In [11]:
index = find_by_ogrn(org_active, ogrn)

In [12]:
print_nko(index)

{
    "OGRN": "1063905027526",
    "address": "236000, г.Калининград, ул.Калужская, д.24-14",
    "dateOGRN": "01.03.2006 0:00:00",
    "form": "Автономная некоммерческая организация",
    "org_name": "Автономная некоммерческая организация \"Психолого-педагогическое Объединение \"Азбука развития\"",
    "org_num": "3914052323",
    "region": "Калининградская область",
    "registry": "Некоммерческие организации",
    "status": "Зарегистрирована"
}


#### Список сайтов, где будем искать

Будем анализировать сайты, которые предлагает Google (первые 10 результатов) на запрос с названием организации _Психолого-педагогическое Объединение "Азбука развития"_. Пусть они будут сохранены в файле #.txt, где # - ОГРН этой организации.

In [13]:
# Функция для чтения ссылок из файла; возвращает список ссылок

def read_urls(ogrn):
    with open(str(ogrn) + '.txt') as urls_file:  # читаем ссылки из файла (%ОГРН%.txt)
        urls = list(urls_file)
    
    for i in range(len(urls)):                    # удаляем переносы строки в конце ссылок
        if urls[i][-1:] == '\n':
            urls[i] = urls[i][:-1]
    
    urls = [i.decode('cp1251') for i in urls]     # переводим ссылки в юникод

    return urls

In [14]:
urls = read_urls(ogrn)

for x in urls:
    print x

http://www.ambercity.ru/компания/азбука-развития-психолого-педагогическое-объединение/2241087
https://vk.com/club21795428
https://2gis.ru/kaliningrad/firm/5630027815196245?queryState=center%2F20.496866%2C54.733537%2Fzoom%2F17
https://razvitie-rebenka.su/club/azbuka-razvitiya-psihologo-pedagogicheskoe-obedinenie-2286/
https://yandex.ru/maps/org/psikhologo_pedagogicheskoye_obyedineniye_azbuka_razvitiya/1269073758
http://www.azbukarazvitya.ru/
https://www.newkaliningrad.ru/catalog/3457456-azbuka-razvitiya.html
http://www.infomesto.com/Калининград/Медицина+и+фармацевтика/АЗБУКА+РАЗВИТИЯ+ПСИХОЛОГО-ПЕДАГОГИЧЕСКОЕ+ОБЪЕДИНЕНИЕ/165FFEBF-1B7C-4672-AEB1-05A877FD935F
http://www.infomesto.com/Калининград/Медицина_Здоровье_Красота/психолого-педагогическое+объединение+"Азбука+развития"/5023766f344752.42072377
http://kaliningrad.yp.ru/detail/id/azbuka_razvitiya_psikhologo_pedagogicheskoe_obedinenie_1871610/


#### Кому принадлежит сайт

Определим место на странице, где будем искать название организации. Имеет смысл это делать внутри тэга `<footer>`, если он есть. Бывает, что вместо него используют `<div>` с классом `footer`, поэтому дополнительно будем искать во всех тэгах, у которых класс (или id) содержит слово `footer`.

In [15]:
import urllib2                 # модуль для обработки url-адресов
from bs4 import BeautifulSoup  # модуль для парсинга страниц
import re                      # модуль для регулярных выражений

In [16]:
# Функция, показывающая, является ли тэг footer'ом
# (или если слово 'footer' присутствует в его атрибутах class или id)

def is_footer(tag):
    
    # ищем тэг <footer>
    if tag.name == 'footer':
        return True
    
    # ищем 'footer' в классе тэга
    if 'class' in tag.attrs:
        if type(tag['class']) == list:
            for x in tag['class']:
                if 'footer' in x:
                    return True
        elif 'footer' in tag['class']:
            return True
    
    # ищем 'footer' в id тэга
    if 'id' in tag.attrs and 'footer' in tag['id']:
        return True
    
    return False

In [17]:
# Функция, возвращающая текст внутри тэгов, похожих на <footer>

def extract_footer(url):
    
    hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 YaBrowser/17.7.1.719 Yowser/2.5 Safari/537.36'}
    req = urllib2.Request(url, headers = hdr)
    html = urllib2.urlopen(req).read()          # загружаем html-код страницы
    soup = BeautifulSoup(html, 'html.parser')   # сохраняем его как объект BeautifulSoup

    for elem in soup.find_all('script'):        # удаляем из кода все тэги <script>, потому что
        elem.decompose()                        # они не обрабатываются функцией get_text() ниже

    footer = soup.find_all(is_footer)      # находим все тэги, похожие на <footer>
    text = ""

    for f in footer:
        text = text + "\n" + f.get_text()  # удаляем всю разметку и сохраняем только текст

    text = re.sub("\n\n+", "\n", text)     # для удобства вывода удаляем лишние переносы строки

    return text

На будущее: можно попробовать не искать в атрибутах class и id, если уже найден тэг `<footer>`.

In [19]:
print extract_footer(urls[4])


РедактироватьУдалитьУдалить ваш отзыв?ДаНет
СохранитьОтменитьПодписатьсяАнонимно
Пользовательское соглашениеОбратная связьПомощьМобильные картыРазмещение в СправочникеРеклама© 2001-2017 ООО «Яндекс»
Пользовательское соглашениеОбратная связьПомощьМобильные картыРазмещение в СправочникеРеклама© 2001-2017 ООО «Яндекс»
Пользовательское соглашение
Обратная связь
Помощь
Мобильные карты
Размещение в СправочникеРеклама
Размещение в Справочнике
Реклама
© 2001-2017 ООО «Яндекс»


In [20]:
# То же, что extract_footer, но найденные блоки 'footer' отделяются
# друг от друга чертами "-----------------------------------------"

def extract_footer_2(url):
    
    html = urllib2.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    for elem in soup.find_all('script'):
        elem.decompose()

    footer = soup.find_all(is_footer)
    text = ""

    for f in footer:
        text = text + "\n" + "-"*60 + "\n" + f.get_text()  # отличие от extract_footer

    text = re.sub("\n\n+", "\n", text)

    return text

In [21]:
print extract_footer_2(urls[4])


------------------------------------------------------------
РедактироватьУдалитьУдалить ваш отзыв?ДаНет
------------------------------------------------------------
СохранитьОтменитьПодписатьсяАнонимно
------------------------------------------------------------
Пользовательское соглашениеОбратная связьПомощьМобильные картыРазмещение в СправочникеРеклама© 2001-2017 ООО «Яндекс»
------------------------------------------------------------
Пользовательское соглашениеОбратная связьПомощьМобильные картыРазмещение в СправочникеРеклама© 2001-2017 ООО «Яндекс»
------------------------------------------------------------
Пользовательское соглашение
------------------------------------------------------------
Обратная связь
------------------------------------------------------------
Помощь
------------------------------------------------------------
Мобильные карты
------------------------------------------------------------
Размещение в СправочникеРеклама
-----------------------------------

---
Можно подумать, что нас интересует только последний "footer", но это не всегда так:

In [22]:
print extract_footer_2(urls[6])


------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
Реклама
О проекте
Контакты
Карта сайта
Статистика
Политика конфиденциальности
Согласие на обработку персональных данных
© 2003-2017 «Новый
Калининград.Ru».
Свидетельство о регистрации СМИ: Эл №
ФС77-43520, выдано Федеральной службой по надзору в сфере связи, информационных технологий и
массовых коммуникаций (Роскомнадзор) 17 января 2011 г.
18+ Данный сайт не предназначен для просмотра лицам младше 18 лет.
Копирование и использование материалов допускается только с разрешения редакции
Адрес: г.Калининград, ул.Гаражная
2, офис 307
Редакция: +7 (4012) 99-21-76 news@newkaliningrad.ru
Реклама: +7 (4012) 31-07-07 reklama@newkaliningrad.ru
Афиша: afisha@newkaliningrad.ru
Общие вопросы: info@newkaliningrad.ru
Материалы с пометкой «Бизнес», «Выборы 2016» и «PR» публикуются на правах рекламы.
 
--------------------

---
#### Название нужной организации

Определим название организации, которое будем искать.

In [23]:
name = org_active[index]['org_name']
print name

Автономная некоммерческая организация "Психолого-педагогическое Объединение "Азбука развития"


Разобьем название организации на токены.

In [24]:
import nltk    # модуль для обработки текста на естественном языке
from nltk.corpus import stopwords
import string

In [25]:
# Функция, фозвращающая список токенов из переданного ей текста

def tokenize(text):
    
    # заменяем дефисы пробелами, чтобы отдельно обрабатывать слова, написанные через дефис
    text = text.replace('-', ' ')
    
    # составляем список токенов (в нижнем регистре)
    tokens = nltk.word_tokenize(text.lower())

    # убираем знаки пунктуации
    tokens = [i for i in tokens if (i[0] not in string.punctuation)]

    # убираем кавычки-ёлочки
    tokens = [i.replace(u'\xab', '').replace(u'\xbb', '') for i in tokens]
    
    # убираем вспомогательные слова (предлоги, союзы и т.д.)
    tokens = [i for i in tokens if (i not in stopwords.words('russian'))]
    
    # удаляем возможные полученные пустые токены
    tokens = [i for i in tokens if (i != '')]

    return tokens

In [26]:
print name + '\n'

name_tokens = tokenize(name)

for x in name_tokens:
    print x

Автономная некоммерческая организация "Психолого-педагогическое Объединение "Азбука развития"

автономная
некоммерческая
организация
психолого
педагогическое
объединение
азбука
развития


In [27]:
# Посмотрим, как происходит такое же разбиение на токены 
# для случайной НКО из зарегистрированных

# n = random.randrange(len(org_active))
name_random = org_active[n]['org_name']

print name_random + '\n'

for x in tokenize(name_random):
    print x

Местная общественная организация в муниципальном районе "Северное Медведково" Северо-Восточной окружной организации г.Москвы ЛДПР

местная
общественная
организация
муниципальном
районе
северное
медведково
северо
восточной
окружной
организации
г.москвы
лдпр


---
#### Аббревиатуры

На веб-странице форма организации может быть указана аббревиатурой, поэтому мы будем пробовать добавлять аббревиатуру к ее названию. Для этого составим аббревиатуры для имеющихся форм организаций.

In [28]:
# Все имеющиеся формы организаций

forms = sorted(forms)
for x in forms:
    print x

Автономная некоммерческая организация
Адвокатские палаты субъектов РФ
Адвокатское бюро
Ассоциация крестьянских (фермерских) хозяйств
Ассоциация экономического развития
Государственная корпорация
Государственно-общественное объединение
Движение
Иные некоммерческие организации
Казачье общество
Коллегия адвокатов
Национально-культурная автономия
Негосударственный пенсионный фонд
Некоммерческий фонд
Некоммерческое партнерство
Нотариальная палата
Общественная организация
Общественно-государственное общественное объединение
Общественное движение
Общественное учреждение
Общественные объединения
Общественный фонд
Община малочисленных народов
Объединение работодателей
Объединения (союз, ассоциация) юридических лиц
Объединения адвокатов
Орган общественной самодеятельности
Организация
Политическая партия
Профессиональный союз
Религиозная организация
Садоводческие, огороднические, дачные и иные некоммерческие товарищества
Садоводческое, огородническое, дачное некоммерческое объединение
Садоводческ

In [29]:
# Посмотрим, как в названии организаций указывается их форма
# и совпадает ли она с тем, что указано в поле 'form'

k = random.randrange(len(org_active))
print json.dumps(org_active[k]['org_name'], ensure_ascii=False), '\n'
print json.dumps(org_active[k]['form'], ensure_ascii=False)

"Чувашская Республиканская общественная организация любительского собаководства \"Зеленая площадка\"" 

"Общественная организация"


In [30]:
# Составим аббревиатуры форм организаций

forms_acronyms = []

for form in forms:
    x = form.replace('-', ' ')
    tokens = nltk.word_tokenize(x)
    tokens = [i for i in tokens if (i[0] not in string.punctuation)]
    
    if len(tokens) > 1:     # аббревиатура будет, только если есть несколько слов        
        acronym = ''
        for y in tokens:
            if len(y) > 1:  # слова из одной буквы не учитываются
                
                acronym = acronym + y[0].upper()   # добавляем (прописную) букву к аббревиатуре
                
        forms_acronyms.append([acronym])    # добавляем аббревиатуру в список аббревиатур
    else:
        forms_acronyms.append([])
    
forms_dict = {forms[i]: forms_acronyms[i] for i in range(len(forms))}

# Полученный словарь forms_dict имеет в качестве ключей формы организаций, 
# а в качестве значений - списки с аббревиатурами для этих форм, то есть
# форма_орг: [аббревиатура]

In [31]:
# Процедура по выводу форм организаций и списков предполагаемых для них аббревиатур

# Аргумент = словарь с ключами в виде форм организаций и значениями в виде списка аббревиатур

def forms_print(forms_dict):
    
    forms_list = sorted(forms_dict.items()[:])  # копируем список

    for i in range(len(forms_list)):        
        print i, forms_list[i][0], '=',

        for x in forms_list[i][1]:
            print x,
        print '\n',
        
    return

In [32]:
# Аббревиатуры, полученные с помощью алгоритма

forms_print(forms_dict)

0 Автономная некоммерческая организация = АНО 
1 Адвокатские палаты субъектов РФ = АПСР 
2 Адвокатское бюро = АБ 
3 Ассоциация крестьянских (фермерских) хозяйств = АКФХ 
4 Ассоциация экономического развития = АЭР 
5 Государственная корпорация = ГК 
6 Государственно-общественное объединение = ГОО 
7 Движение = 
8 Иные некоммерческие организации = ИНО 
9 Казачье общество = КО 
10 Коллегия адвокатов = КА 
11 Национально-культурная автономия = НКА 
12 Негосударственный пенсионный фонд = НПФ 
13 Некоммерческий фонд = НФ 
14 Некоммерческое партнерство = НП 
15 Нотариальная палата = НП 
16 Общественная организация = ОО 
17 Общественно-государственное общественное объединение = ОГОО 
18 Общественное движение = ОД 
19 Общественное учреждение = ОУ 
20 Общественные объединения = ОО 
21 Общественный фонд = ОФ 
22 Община малочисленных народов = ОМН 
23 Объединение работодателей = ОР 
24 Объединения (союз, ассоциация) юридических лиц = ОСАЮЛ 
25 Объединения адвокатов = ОА 
26 Орган общественной само

In [33]:
# Подредактируем некоторые аббревиатуры вручную,
# пользуясь номерами элементов в списке forms_list

forms_list = sorted(forms_dict.items()[:])

forms_dict[forms_list[1][0]]   = [u'АПСРФ']
forms_dict[forms_list[3][0]]  += [u'АКХ', u'АФХ']
forms_dict[forms_list[8][0]]   = [u'НО', u'НКО']
forms_dict[forms_list[24][0]] += [u'ОЮЛ', u'СЮЛ', u'АЮЛ']
forms_dict[forms_list[31][0]] += [u'СНТ', u'ОНТ', u'ДНТ', u'НТ']
forms_dict[forms_list[32][0]] += [u'СНО', u'ОНО', u'ДНО', u'НО']
forms_dict[forms_list[33][0]] += [u'СНП', u'ОНП', u'ДНП', u'НП']
forms_dict[forms_list[35][0]] += [u'СОО', u'АОО']

In [34]:
# Для полноты добавим также вариации некоторых форм организаций

forms_dict[u'Ассоциация крестьянских хозяйств'] = [u'АКХ']
forms_dict[u'Ассоциация фермерских хозяйств'] = [u'АФХ']
forms_dict[u'Объединение юридических лиц'] = [u'ОЮЛ']
forms_dict[u'Союз юридических лиц'] = [u'СЮЛ']
forms_dict[u'Ассоциация юридических лиц'] = [u'АЮЛ']
forms_dict[u'Садоводческое некоммерческое товарищество'] = [u'СНТ', u'НТ']
forms_dict[u'Огородническое некоммерческое товарищество'] = [u'ОНТ', u'НТ']
forms_dict[u'Дачное некоммерческое товарищество'] = [u'ДНТ', u'НТ']
forms_dict[u'Садоводческое некоммерческое объединение'] = [u'СНО', u'НО']
forms_dict[u'Огородническое некоммерческое объединение'] = [u'ОНО', u'НО']
forms_dict[u'Дачное некоммерческое объединение'] = [u'ДНО', u'НО']
forms_dict[u'Садоводческое некоммерческое партнерство'] = [u'СНП', u'НП']
forms_dict[u'Огородническое некоммерческое партнерство'] = [u'ОНП', u'НП']
forms_dict[u'Дачное некоммерческое партнерство'] = [u'ДНП', u'НП']
forms_dict[u'Союз общественных объединений'] = [u'СОО']
forms_dict[u'Ассоциация общественных объединений'] = [u'АОО']

In [35]:
# Исправленные аббревиатуры

forms_print(forms_dict)

0 Автономная некоммерческая организация = АНО 
1 Адвокатские палаты субъектов РФ = АПСРФ 
2 Адвокатское бюро = АБ 
3 Ассоциация крестьянских (фермерских) хозяйств = АКФХ АКХ АФХ 
4 Ассоциация крестьянских хозяйств = АКХ 
5 Ассоциация общественных объединений = АОО 
6 Ассоциация фермерских хозяйств = АФХ 
7 Ассоциация экономического развития = АЭР 
8 Ассоциация юридических лиц = АЮЛ 
9 Государственная корпорация = ГК 
10 Государственно-общественное объединение = ГОО 
11 Дачное некоммерческое объединение = ДНО НО 
12 Дачное некоммерческое партнерство = ДНП НП 
13 Дачное некоммерческое товарищество = ДНТ НТ 
14 Движение = 
15 Иные некоммерческие организации = НО НКО 
16 Казачье общество = КО 
17 Коллегия адвокатов = КА 
18 Национально-культурная автономия = НКА 
19 Негосударственный пенсионный фонд = НПФ 
20 Некоммерческий фонд = НФ 
21 Некоммерческое партнерство = НП 
22 Нотариальная палата = НП 
23 Общественная организация = ОО 
24 Общественно-государственное общественное объединение = 

In [36]:
# Добавим в список токенов аббревиатуру, если есть подходящая

name_tokens = tokenize(name)   # на всякий случай повторим токенизацию

for x in forms_dict.keys():
    if x.lower() in name.lower():
        name_tokens += forms_dict[x]

for x in name_tokens:
    print x

автономная
некоммерческая
организация
психолого
педагогическое
объединение
азбука
развития
АНО


---
#### Поиск названия организации

Теперь определим, какая из обработанных веб-страниц содержит в `<footer>` название данной организации.

In [37]:
print name

Автономная некоммерческая организация "Психолого-педагогическое Объединение "Азбука развития"


In [38]:
# Составим из списка токенов множество

name_set = set(name_tokens)

In [39]:
# Процедура, которая находит на веб-страницах из urls слова из name_set
# и печатает их

def name_search(urls, name_set):
    
    for i in range(len(urls)):

        # составим множество слов, в котором будем искать
        candidate = extract_footer(urls[i].encode('cp1251'))
        candidate_set = set(tokenize(candidate))

        # найдем пересечение множеств
        match = list(candidate_set & name_set)

        print str(i) + ':'
        for x in match:
            print '\t' + x

In [40]:
name_search(urls, name_set)

0:
1:
2:
3:
	развития
4:
5:
	развития
	азбука
6:
7:
8:
9:


---

Повторим поиск названия организации на сайтах для другой НКО.

In [42]:
# Функция, подготавливающая название организации для поиска; возвращает множество слов из названия

def name_prep(npo_list, index):
    
    name = npo_list[index]['org_name']   # извлекаем название орг. из списка по ее индексу
    name_tokens = tokenize(name)         # разбиваем название на токены
    
    for x in forms_dict.keys():          # по возможности добавляем аббревиатуру к токенам
        if x.lower() in name.lower():
            name_tokens += forms_dict[x]
    
    return set(name_tokens)              # возвращаем множество из токенов

In [48]:
ogrn_1 = 1027100005283

index_1 = find_by_ogrn(org_active, ogrn_1)

name_set_1 = name_prep(org_active, index_1)

urls_1 = read_urls(ogrn_1)

print json.dumps(org_active[index_1]['org_name'], ensure_ascii=False), '\n'

name_search(urls_1, name_set_1)

"Местная религиозная организация прихода Рождества Пресвятой Богородицы Римско-католической Церкви города Тулы" 

0:
1:
2:
3:
4:
5:
6:
7:
8:
9:


In [49]:
for url in urls1:
    print extract_footer(url.encode('cp1251'))
    print '-'*60


Закрыть
Закрыть
Закрыть
Закрыть
Самый популярный портал проверки контрагентов в РФ
*по данным на 01.07.2017 Рамблер ТОП 100
Статьи 
Проверка контрагента
Проверка работодателя
Поддержка бизнеса
Руководитель
Учредители
Акционеры
Аффилированность
Открытые данные
Филиалы
Виды деятельности
Бухгалтерская отчетность
Арбитражный суд
Дата регистрации
Юридический адрес
Лицензии
Налоговый орган
ОКПО
Реквизиты организации
ПФР
ФСС
Статус
ЕГРЮЛ
ЕГРИП
ОГРН
ОГРНИП
ИНН
КПП
© ЗАЧЕСТНЫЙБИЗНЕС. Правовая информация.Использование сайта означает согласие с правилами.
Политика в отношении обработки персональных данных
Свидетельство о регистрации СМИ ЭЛ № ФС77-70272 выдано Федеральной службой по надзору в сфере связи, информационных технологий и массовых коммуникаций (Роскомнадзор) 10.07.2017 г. 18+
Зарегистрироваться
Восстановить пароль

------------------------------------------------------------

------------------------------------------------------------

      Если вы нашли на этом сайте полезную для се